# Note:
This model was not tested due to poor performances in comparison with the Ontonotes model from Flair.

In [1]:
import spacy
import csv
from tqdm import tqdm
import re

C:\Users\CSA\Anaconda3\envs\ner_vasari\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("../vasari-kg.github.io/data/sentences_en.csv", "r") as f:
    sentences = list(csv.DictReader(f=f, delimiter=","))

In [8]:
nlp = spacy.load("en_core_web_trf")

In [10]:
def convert_label(label):
    if label == "PERSON":
        return "PER"
    if label == "ORGANIZATION":
        return "ORG"
    if label in {"GPE", "FAC"}:
        return "LOC"
    if label =="DATE":
        return "DATE"
    else:
        return "MISC"

In [11]:
output = []

pbar = tqdm(total=len(sentences))
for sample in sentences:
    sent_idx = sample["id"]
    text = sample["sentence"]
    doc = nlp(text)
    for ent in doc.ents:
        start_pos = ent.start_char
        end_pos = ent.end_char
        surface = ent.text
        ner_type = ent.label_
        match = re.match('(?:^the\s|^a\s)(.*?)$', surface, re.IGNORECASE)
        if match:
            surface = match.group(1)
            start_pos = end_pos - len(surface)
        if ner_type not in {"TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL","CARDINAL"}:
            ner_type = convert_label(ner_type)
            output.append({
                "id":sent_idx,
                "start_pos":start_pos,
                "end_pos":end_pos,
                "surface":surface,
                "type":ner_type
            })
    pbar.update(1)
pbar.close()


keys = output[0].keys()
a_file = open("results3/spacy_en/output.csv", "w")
dict_writer = csv.DictWriter(a_file, keys)
dict_writer.writeheader()
dict_writer.writerows(output)
a_file.close()
    

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:09<00:00,  4.17it/s]


beautiful mind
